In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pandas
import pandas as pd

!pip install titlecase
import titlecase

from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# references
state_names = ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
state_names = [titlecase.titlecase(x) for x in state_names]
state_abbrevs = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

state_to_abbrev_map = {}

for x in zip(state_names, state_abbrevs):
    state_name = x[0]
    abbrev = x[1]
    state_to_abbrev_map[state_name] = abbrev

In [3]:
inflation = pd.read_csv("https://raw.githubusercontent.com/voglerdaniel/DATA-511-Final-Project/main/Inflation%20Rates%20v2.csv")
unemployment = pd.read_csv("https://github.com/voglerdaniel/DATA-511-Final-Project/blob/46fc9585e080e1d9a28217db67e552e78a81b3f8/Historical%20Unemployment%20Rates%20by%20City.csv?raw=true")
hpi = pd.read_csv("https://github.com/voglerdaniel/DATA-511-Final-Project/blob/46fc9585e080e1d9a28217db67e552e78a81b3f8/HPI_clean.csv?raw=true")
income_data = pd.read_csv("https://github.com/ksorstokke2/data511/blob/99147390eb64e2458ed97cce5ea9c438f47b89c7/incomeMSA.csv?raw=true")
disability = pd.read_csv("https://raw.githubusercontent.com/sarahnguyen7/UW-DATA_511-FinalProj/main/PublicHealth_Disability_cleaned_updated.csv")
uninsured = pd.read_csv("https://raw.githubusercontent.com/sarahnguyen7/UW-DATA_511-FinalProj/main/PublicHealth_Uninsured_cleaned_updated.csv")
homeless_rate = pd.read_csv("https://github.com/voglerdaniel/DATA-511-Final-Project/blob/46fc9585e080e1d9a28217db67e552e78a81b3f8/Final-Homelessness-Rate-Data%20-%20Final-Homelessness-Rate-Data.csv?raw=true")
demographics = pd.read_csv("https://raw.githubusercontent.com/parvatijay2901/Homelessness-in-the-US/main/demographic-data_cleaned_city_120320231203.csv")
snap = pd.read_csv("https://raw.githubusercontent.com/lawrieb/DATA-511-Final-Project/main/out_snap.csv")
economic = pd.read_csv("https://raw.githubusercontent.com/lawrieb/DATA-511-Final-Project/main/out_economic.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/lawrieb/DATA-511-Final-Project/main/out_deaths.csv")
rentals = pd.read_csv("https://raw.githubusercontent.com/voglerdaniel/DATA-511-Final-Project/main/Rental%20Data.csv")
safety = pd.read_csv("https://raw.githubusercontent.com/manya28/Safety-Net-Cleaned-Data/main/safety-net-final.csv")


datasets = {"Inflation": inflation,
            "Unemployment": unemployment,
            "HPI": hpi,
            "Median Income": income_data,
            "Disability": disability,
            "uninsured": uninsured,
            "HL Rate (Output)": homeless_rate,
            "Demographics": demographics,
            "Snap": snap,
            "Economic": economic,
            "Deaths": deaths,
            "Rental Prices": rentals,
            "No. of Beds": safety}

In [4]:
def test(df):

    if not ("MSA" in df.columns.values and "Year" in df.columns.values):

        return "Dataset does not have columns named MSA and Year as required"

    msas = list(df["MSA"].values)
    years = list(df["Year"].values)

    msa_year = [x for x in zip(msas, years)]

    unique_msa_year = set(msa_year)

    if len(msa_year) == len(unique_msa_year):

        return True

    else:

        return False

In [5]:
for x in datasets.keys():
  print(x, ": ", test(datasets[x]))

Inflation :  True
Unemployment :  True
HPI :  True
Median Income :  True
Disability :  True
uninsured :  True
HL Rate (Output) :  False
Demographics :  Dataset does not have columns named MSA and Year as required
Snap :  True
Economic :  True
Deaths :  True
Rental Prices :  True
No. of Beds :  True


## Step 1: Test that all Datasets Pass the QC Test
#### (Each MSA-Year Combo Appears Once)

## Step 2: Clean Up the Datasets to Prepare them for Merge

In [6]:
inflation["Primary MSA City"] = [str(x).split("-")[0] for x in inflation["MSA"].values]
inflation["Primary MSA City"] = [str(x).split(",")[0] for x in inflation["Primary MSA City"].values]

primary_msa_state = []

for x in inflation["MSA"]:
    try:
        state = x.split(",")[1].split("-")[0].split("(")[0]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" (CBSA)")[0][-2:])


inflation["Primary MSA State"] = primary_msa_state
inflation = inflation.applymap(lambda x: x.strip() if isinstance(x, str) else x)

inflation

,MSA,Year,Inflation Rate over Previous Year,Primary MSA City,Primary MSA State
0,"San Francisco-Oakland-Hayward, CA (CBSA)",2003,0.017868,San Francisco,CA
1,"New York-Newark-Jersey City, NY-NJ-PA (CBSA)",2003,0.030702,New York,NY
2,"Dallas-Fort Worth-Arlington, TX (CBSA)",2003,0.021057,Dallas,TX
3,"Chicago-Naperville-Elgin, IL-IN-WI (CBSA)",2003,0.018398,Chicago,IL
4,"Boston-Cambridge-Newton, MA-NH (CBSA)",2003,0.037024,Boston,MA
...,...,...,...,...,...
575,"Cleveland-Akron, OH (CMSA) (DISCONTINUED)",2022,NaN,Cleveland,OH
576,"St. Louis, MO-IL (CBSA)",2022,0.077591,St. Louis,MO
577,"Kansas City, MO-KS (CMSA) (DISCONTINUED)",2022,NaN,Kansas City,MO
578,"Pittsburgh, PA (CMSA) (DISCONTINUED)",2022,NaN,Pittsburgh,PA


In [7]:
set(inflation["Primary MSA City"].values)

{'Atlanta',
 'Boston',
 'Chicago',
 'Cincinnati',
 'Cleveland',
 'Dallas',
 'Denver',
 'Detroit',
 'Houston',
 'Kansas City',
 'Los Angeles',
 'Miami',
 'Milwaukee',
 'Minneapolis',
 'New York',
 'Philadelphia',
 'Phoenix',
 'Pittsburgh',
 'Portland',
 'San Diego',
 'San Francisco',
 'Seattle',
 'St. Louis',
 'Tampa',
 'Urban Alaska (CBSA)',
 'Urban Hawaii (CBSA)',
 'Washington'}

In [8]:
snap["Primary MSA City"] = [str(x).split("-")[0] for x in snap["MSA"].values]
snap["Primary MSA City"] = [str(x).split(",")[0] for x in snap["Primary MSA City"].values]

primary_msa_state = []

for x in snap["MSA"]:
    try:
        state = x.split(",")[1][1:3]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" MSA")[0][-2:])


snap["Primary MSA State"] = primary_msa_state
snap = snap.applymap(lambda x: x.strip() if isinstance(x, str) else x)

snap

,MSA,Year,nSNAP,Primary MSA City,Primary MSA State
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2007,2305827,Atlanta,GA
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2008,2715951,Atlanta,GA
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2009,3377151,Atlanta,GA
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2010,3855078,Atlanta,GA
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2011,4221267,Atlanta,GA
...,...,...,...,...,...
681,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2016,3416790,Washington,DC
682,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2017,3180450,Washington,DC
683,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2018,3027705,Washington,DC
684,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2019,2990490,Washington,DC


In [9]:
deaths["Primary MSA City"] = [str(x).split("-")[0] for x in deaths["MSA"].values]
deaths["Primary MSA City"] = [str(x).split(",")[0] for x in deaths["Primary MSA City"].values]

primary_msa_state = []

for x in deaths["MSA"]:
    try:
        state = x.split(",")[1][1:3]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" MSA")[0][-2:])


deaths["Primary MSA State"] = primary_msa_state
deaths = deaths.applymap(lambda x: x.strip() if isinstance(x, str) else x)

deaths

,MSA,Year,Population,Overdose Deaths,Unintentional Overdose Deaths,Alcohol-Induced Deaths,Primary MSA City,Primary MSA State
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2007,1550291,155,26,57,Atlanta,GA
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2008,1574340,171,31,66,Atlanta,GA
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2009,1596169,133,40,40,Atlanta,GA
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2010,1612474,166,46,107,Atlanta,GA
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2011,1649492,156,61,96,Atlanta,GA
...,...,...,...,...,...,...,...,...
681,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2016,3046837,527,452,182,Washington,DC
682,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2017,3098508,577,505,185,Washington,DC
683,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2018,2965632,471,397,185,Washington,DC
684,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2019,3174509,569,497,198,Washington,DC


In [10]:
economic["Primary MSA City"] = [str(x).split("-")[0] for x in economic["MSA"].values]
economic["Primary MSA City"] = [str(x).split(",")[0] for x in economic["Primary MSA City"].values]

primary_msa_state = []

for x in economic["MSA"]:
    try:
        state = x.split(",")[1][1:3]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" MSA")[0][-2:])


economic["Primary MSA State"] = primary_msa_state
economic = economic.applymap(lambda x: x.strip() if isinstance(x, str) else x)

economic

,MSA,Year,Population (persons) 3/,Total employment (number of jobs),Average earnings per job (dollars),Average wages and salaries (dollars),Per capita net earnings 4/ (dollars),Primary MSA City,Primary MSA State
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2007,1550291,1377965,60308.50,54231.50,36294.00,Atlanta,GA
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2008,1574340,1373823,58787.50,54938.50,33614.00,Atlanta,GA
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2009,1596169,1316341,59742.50,55239.50,33045.50,Atlanta,GA
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2010,1618043,1314438,60631.00,56502.50,33518.50,Atlanta,GA
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",2011,1645464,1356683,61490.50,58142.50,35332.00,Atlanta,GA
...,...,...,...,...,...,...,...,...,...
779,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2018,1486036,1524121,91120.50,84751.50,62675.75,Washington,DC
780,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2019,1499021,1535453,93425.25,86451.25,63774.75,Washington,DC
781,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2020,1491461,1471123,98979.25,93441.50,64841.25,Washington,DC
782,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",2021,1486003,1489799,103436.50,97194.50,71346.00,Washington,DC


In [11]:
disability["Primary MSA City"] = [str(x).split("-")[0] for x in disability["MSA"].values]
disability["Primary MSA City"] = [str(x).split(",")[0] for x in disability["Primary MSA City"].values]

primary_msa_state = []

for x in disability["MSA"]:
    try:
        state = x.split(",")[1][1:3]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" (CBSA)")[0][-2:])


disability["Primary MSA State"] = primary_msa_state
disability = disability.applymap(lambda x: x.strip() if isinstance(x, str) else x)

disability

,MSA,Year,Percent With A Disability,Primary MSA City,Primary MSA State
0,"Atlanta-Sandy Springs-Marietta, GA",2010,9.1,Atlanta,GA
1,"Augusta-Richmond County, GA-SC",2010,14.5,Augusta,GA
2,"Austin-Round Rock-San Marcos, TX",2010,9.1,Austin,TX
3,"Baltimore-Towson, MD",2010,10.9,Baltimore,MD
4,"Birmingham-Hoover, AL",2010,15.3,Birmingham,AL
...,...,...,...,...,...
661,"San Jose-Sunnyvale-Santa Clara, CA",2022,9.3,San Jose,CA
662,"Seattle-Tacoma-Bellevue, WA",2022,11.8,Seattle,WA
663,"Tampa-St. Petersburg-Clearwater, FL",2022,14.5,Tampa,FL
664,"Virginia Beach-Norfolk-Newport News, VA-NC",2022,14.4,Virginia Beach,VA


In [12]:
uninsured["Primary MSA City"] = [str(x).split("-")[0] for x in uninsured["MSA"].values]
uninsured["Primary MSA City"] = [str(x).split(",")[0] for x in uninsured["Primary MSA City"].values]

primary_msa_state = []

for x in uninsured["MSA"]:
    try:
        state = x.split(",")[1].split("-")[0].split("(")[0]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" (CBSA)")[0][-2:])


uninsured["Primary MSA State"] = primary_msa_state
uninsured = uninsured.applymap(lambda x: x.strip() if isinstance(x, str) else x)

uninsured

,MSA,Year,Uninsured Population,Total Population,Ratio of Uninsured,Percentage of Uninsured,Primary MSA City,Primary MSA State
0,"Atlanta-Sandy Springs-Marietta, GA",2010,1041340,5244660,0.198552,19.855243,Atlanta,GA
1,"Augusta-Richmond County, GA-SC",2010,89738,541705,0.165658,16.565843,Augusta,GA
2,"Austin-Round Rock-San Marcos, TX",2010,330058,1710245,0.192989,19.298872,Austin,TX
3,"Baltimore-Towson, MD",2010,268069,2665963,0.100552,10.055241,Baltimore,MD
4,"Birmingham-Hoover, AL",2010,151717,1117020,0.135823,13.582299,Birmingham,AL
...,...,...,...,...,...,...,...,...
557,"San Francisco-Oakland-Berkeley, CA",2022,175372,4550597,0.038538,3.853824,San Francisco,CA
558,"Seattle-Tacoma-Bellevue, WA",2022,211515,3980345,0.053140,5.313987,Seattle,WA
559,"Tampa-St. Petersburg-Clearwater, FL",2022,348049,3255447,0.106913,10.691281,Tampa,FL
560,"Virginia Beach-Norfolk-Newport News, VA-NC",2022,107528,1700944,0.063217,6.321666,Virginia Beach,VA


In [13]:
unemployment["Primary MSA City"] = [str(x).split("-")[0] for x in unemployment["MSA"].values]
unemployment["Primary MSA City"] = [str(x).split(",")[0] for x in unemployment["Primary MSA City"].values]

primary_msa_state = []

for x in unemployment["MSA"]:

    try:
        state = x.split(",")[1].split("-")[0].split("(")[0]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append(x.split(" (MSA)")[0][-2:])

unemployment["Primary MSA State"] = primary_msa_state
unemployment = unemployment.applymap(lambda x: x.strip() if isinstance(x, str) else x)

unemployment

,MSA,Year,Unemployment Rate,Primary MSA City,Primary MSA State
0,"Dallas-Fort Worth-Arlington, TX (MSA)",1990,5.2,Dallas,TX
1,"Seattle-Tacoma-Bellevue, WA (MSA)",1990,4.0,Seattle,WA
2,"Austin-Round Rock, TX (MSA)",1990,4.9,Austin,TX
3,"Chicago-Naperville-Elgin, IL-IN-WI (MSA)",1990,6.4,Chicago,IL
4,"St. Louis, MO-IL (MSA)",1990,6.0,St. Louis,MO
...,...,...,...,...,...
1777,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD (MSA)",2022,4.1,Philadelphia,PA
1778,"Buffalo-Cheektowaga-Niagara Falls, NY (MSA)",2022,3.6,Buffalo,NY
1779,"Visalia-Porterville, CA (MSA)",2022,8.2,Visalia,CA
1780,"Salt Lake City, UT (MSA)",2022,2.3,Salt Lake City,UT


In [14]:
hpi["Primary MSA City"] = [str(x).split("-")[0] for x in hpi["MSA"].values]
hpi["Primary MSA City"] = [str(x).split(",")[0] for x in hpi["Primary MSA City"].values]

primary_msa_state = []

for x in hpi["MSA"].values:

    try:
        state = str(x).split(",")[1].split("-")[0].split("(")[0][1:]
        primary_msa_state.append(state)
    except:
        primary_msa_state.append("N/A")

hpi["Primary MSA State"] = primary_msa_state
hpi = hpi.applymap(lambda x: x.strip() if isinstance(x, str) else x)
hpi

,MSA,Year,HPI (Not Seasonally Adjusted),Primary MSA City,Primary MSA State
0,"Abilene, TX",1986,103.1200,Abilene,TX
1,"Abilene, TX",1987,97.3725,Abilene,TX
2,"Abilene, TX",1988,84.2000,Abilene,TX
3,"Abilene, TX",1989,82.2525,Abilene,TX
4,"Abilene, TX",1990,80.4175,Abilene,TX
...,...,...,...,...,...
19585,"Yuma, AZ",2019,193.8625,Yuma,AZ
19586,"Yuma, AZ",2020,207.7450,Yuma,AZ
19587,"Yuma, AZ",2021,249.5150,Yuma,AZ
19588,"Yuma, AZ",2022,303.6675,Yuma,AZ


In [15]:
# Clean income data

# clean cities
income_data["State"] = [x.strip() for x in income_data["State"].values]
income_data["City"] = [x.split(" city")[0] for x in income_data["City"].values]
income_data["City"] = [x.split(" urbana")[0] for x in income_data["City"].values]
income_data["City"] = [x.split(" CDP")[0] for x in income_data["City"].values]
income_data["City"] = [x.split(" municipality")[0] for x in income_data["City"].values]
income_data["City"] = [x.split(" zona")[0] for x in income_data["City"].values]
income_data["City"].replace({"Nashville-Davidson metropolitan government (balance)":
                             "Nashville", "Louisville/Jefferson County metro government (balance)":
                             "Louisville"}, inplace=True)


data_states = []

for x in income_data["State"].values:

    if x in state_to_abbrev_map.keys():
        data_states.append(state_to_abbrev_map[x])

    elif x == "District of Columbia":
        data_states.append("DC")

    else:
        data_states.append("N/A")


income_data["State"] = data_states

income_data = income_data.rename(columns={"Income": "Median Income ($)"})
income_data.drop(columns=["Unnamed: 0"],inplace=True)
income_data.rename(columns={"City": "Primary MSA City", "State": "Primary MSA State"}, inplace=True)
income_data

,Primary MSA City,Primary MSA State,Median Income ($),Year,MSA
0,Atlanta,GA,83251,2022,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
1,Atlanta,GA,50210,2015,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
2,Atlanta,GA,65345,2018,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
3,Atlanta,GA,46485,2013,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
4,Atlanta,GA,46466,2012,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
...,...,...,...,...,...
583,Washington,DC,75506,2016,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"
584,Washington,DC,85203,2018,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"
585,Washington,DC,82372,2017,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"
586,Washington,DC,67572,2013,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"


In [16]:
homeless_rate.rename(columns={"City": "Primary MSA City", "State": "Primary MSA State"}, inplace=True)
homeless_rate = homeless_rate[["MSA", "Primary MSA City", "Primary MSA State", "Year", "Overall Homeless", "Population", "Rate of Homelessness"]]

city_populations = set(zip(homeless_rate["MSA"].values, homeless_rate["Year"].values, homeless_rate["Population"].values))
city_populations = pd.DataFrame(columns=["MSA", "Year", "Population"], data=city_populations)

homeless_rate = homeless_rate.drop(columns = ["Population", "Rate of Homelessness"])
homeless_rate = homeless_rate.groupby(by=["MSA", "Primary MSA City", "Primary MSA State", "Year"]).agg('sum').reset_index()

populations_for_table = []

for index, row in homeless_rate.iterrows():
    msa = row["MSA"]
    year = row["Year"]
    populations_for_table.append(city_populations[(city_populations["MSA"] == msa) & (city_populations["Year"] == year)]["Population"].values[0])

homeless_rate["Population"] = populations_for_table
homeless_rate["Homeless Rate (Percentage of Population)"] = (homeless_rate["Overall Homeless"] / homeless_rate["Population"])*100

test(homeless_rate)

True

In [17]:
uninsured = uninsured[['MSA', 'Year', 'Percentage of Uninsured', 'Primary MSA City', 'Primary MSA State']]
uninsured

,MSA,Year,Percentage of Uninsured,Primary MSA City,Primary MSA State
0,"Atlanta-Sandy Springs-Marietta, GA",2010,19.855243,Atlanta,GA
1,"Augusta-Richmond County, GA-SC",2010,16.565843,Augusta,GA
2,"Austin-Round Rock-San Marcos, TX",2010,19.298872,Austin,TX
3,"Baltimore-Towson, MD",2010,10.055241,Baltimore,MD
4,"Birmingham-Hoover, AL",2010,13.582299,Birmingham,AL
...,...,...,...,...,...
557,"San Francisco-Oakland-Berkeley, CA",2022,3.853824,San Francisco,CA
558,"Seattle-Tacoma-Bellevue, WA",2022,5.313987,Seattle,WA
559,"Tampa-St. Petersburg-Clearwater, FL",2022,10.691281,Tampa,FL
560,"Virginia Beach-Norfolk-Newport News, VA-NC",2022,6.321666,Virginia Beach,VA


In [18]:
safety

,MSA,Year,Primary City in MSA,Primary State in MSA,Total Year-Round Beds (ES+TH)
0,"Providence-Warwick, RI-MA MSA",2011,Providence,MA,1714
1,"San Antonio-New Braunfels, TX MSA",2011,San Antonio,TX,2302
2,"Birmingham-Hoover, AL MSA",2012,Birmingham,AL,1097
3,"San Jose-Sunnyvale-Santa Clara, CA MSA",2010,San Jose,CA,1783
4,"San Diego-Chula Vista-Carlsbad, CA MSA",2017,San Diego,CA,3680
...,...,...,...,...,...
779,"Orlando-Kissimmee-Sanford, FL MSA",2015,Orlando,FL,2155
780,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA",2020,Philadelphia,PA,9439
781,"Denver-Aurora-Lakewood, CO MSA",2009,Denver,CO,4318
782,"Los Angeles-Long Beach-Anaheim, CA MSA",2010,Los Angeles,CA,17973


In [19]:
rentals.rename(columns={"Unnamed: 2": "Primary State in MSA"}, inplace=True)
rentals

,MSA,Primary City in MSA,Primary State in MSA,Year,Studio Rental Price,1BR Rental Price,2BR Rental Price,3BR Rental Price,4BR Rental Price
0,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",Atlanta,GA,2022,1205,1226,1397,1730,2115
1,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",Atlanta,GA,2021,1101,1126,1284,1615,1976
2,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",Atlanta,GA,2020,1065,1096,1248,1592,1951
3,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",Atlanta,GA,2019,1005,1031,1180,1522,1870
4,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",Atlanta,GA,2018,930,957,1099,1432,1760
...,...,...,...,...,...,...,...,...,...
539,"Tampa-St. Petersburg-Clearwater, FL MSA",Tampa,FL,2010,758,848,1021,1315,1603
540,"Tampa-St. Petersburg-Clearwater, FL MSA",Tampa,FL,2009,748,838,1008,1298,1583
541,"Tampa-St. Petersburg-Clearwater, FL MSA",Tampa,FL,2008,698,782,941,1212,1477
542,"Tampa-St. Petersburg-Clearwater, FL MSA",Tampa,FL,2007,647,725,872,1123,1369


In [20]:
demographics.rename({'State':'Primary MSA State'}, axis=1, inplace=True)
demographics.head(5)

,Year,Primary MSA City,Primary MSA State,Population,Sheltered Homeless_percentage,Sheltered Homeless - Under 18_percentage,Sheltered Homeless - Age 18 to 24_percentage,Sheltered Homeless - Over 24_percentage,Sheltered Homeless - Female_percentage,Sheltered Homeless - Male_percentage,...,Overall Homeless - Male_percentage,Overall Homeless - Transgender_percentage,Overall Homeless - Gender Non-Conforming_percentage,Overall Homeless - Hispanic/Latino_percentage,Overall Homeless - White_percentage,Overall Homeless - Black or African American_percentage,Overall Homeless - Asian_percentage,Overall Homeless - American Indian or Alaska Native_percentage,Overall Homeless - Native Hawaiian or Other Pacific Islander_percentage,Overall Homeless - Multiple Races_percentage
0,2021,Washington,DC,6364907,0.096294,0.018461,0.009568,0.068265,0.038068,0.057283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021,Dallas,TX,7773289,0.050931,0.007423,0.003718,0.039790,0.017637,0.033152,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021,Memphis,TN,1335407,0.040212,0.007413,0.001872,0.030927,0.012131,0.028081,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021,Los Angeles,CA,12972847,0.166633,0.045988,0.014746,0.105898,0.076984,0.088099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021,Las Vegas,NV,2295194,0.099730,0.009106,0.008627,0.081997,0.028799,0.070190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Merge Datasets

#### Merging homeless rate (output) and inflation data

In [21]:
df = pd.merge(inflation, homeless_rate, on=["Primary MSA City", "Primary MSA State", "Year"], how='right')
df = df[["MSA_y", "Primary MSA City", "Primary MSA State", "Year", "Inflation Rate over Previous Year", "Overall Homeless", "Population", "Homeless Rate (Percentage of Population)"]]
df = df.rename(columns={"Inflation Rate over Previous Year": "Inflation Rate", "MSA_y": "MSA"})
df["Inflation Rate"] = round(df["Inflation Rate"]*100,1)
df

,MSA,Primary MSA City,Primary MSA State,Year,Inflation Rate,Overall Homeless,Population,Homeless Rate (Percentage of Population)
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2007,3.2,6840,4939198,0.138484
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2008,2.9,6840,5046727,0.135533
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2009,-2.5,7019,5156598,0.136117
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2010,1.2,7019,5268860,0.133217
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2011,2.9,6805,5366462,0.126806
...,...,...,...,...,...,...,...,...
760,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2017,1.3,9393,6211341,0.151223
761,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2018,NaN,8846,6247841,0.141585
762,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2019,NaN,8415,6280487,0.133986
763,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2020,NaN,8332,6366679,0.130869


#### Merging data from above with unemployment

In [22]:
df = pd.merge(df, unemployment, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df = df[["MSA_x", "Primary MSA City", "Primary MSA State", "Year", "Inflation Rate", "Unemployment Rate", "Overall Homeless", "Population", "Homeless Rate (Percentage of Population)"]]
df = df.rename(columns={"MSA_x": "MSA"})
df

,MSA,Primary MSA City,Primary MSA State,Year,Inflation Rate,Unemployment Rate,Overall Homeless,Population,Homeless Rate (Percentage of Population)
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2007,3.2,4.8,6840,4939198,0.138484
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2008,2.9,6.7,6840,5046727,0.135533
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2009,-2.5,10.4,7019,5156598,0.136117
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2010,1.2,10.4,7019,5268860,0.133217
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2011,2.9,9.8,6805,5366462,0.126806
...,...,...,...,...,...,...,...,...,...
760,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2017,1.3,3.6,9393,6211341,0.151223
761,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2018,NaN,3.3,8846,6247841,0.141585
762,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2019,NaN,3.0,8415,6280487,0.133986
763,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2020,NaN,6.4,8332,6366679,0.130869


#### Merge previous data with HPI data

In [23]:
df = pd.merge(df, hpi, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df = df[["MSA_x", "Primary MSA City", "Primary MSA State", "Year", "Inflation Rate", "Unemployment Rate", "HPI (Not Seasonally Adjusted)", "Overall Homeless", "Population", "Homeless Rate (Percentage of Population)"]]
df = df.rename(columns={"MSA_x": "MSA"})
df

,MSA,Primary MSA City,Primary MSA State,Year,Inflation Rate,Unemployment Rate,HPI (Not Seasonally Adjusted),Overall Homeless,Population,Homeless Rate (Percentage of Population)
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2007,3.2,4.8,181.4050,6840,4939198,0.138484
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2008,2.9,6.7,174.9125,6840,5046727,0.135533
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2009,-2.5,10.4,164.4725,7019,5156598,0.136117
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2010,1.2,10.4,151.7375,7019,5268860,0.133217
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2011,2.9,9.8,140.0650,6805,5366462,0.126806
...,...,...,...,...,...,...,...,...,...,...
760,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2017,1.3,3.6,260.1925,9393,6211341,0.151223
761,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2018,NaN,3.3,268.7350,8846,6247841,0.141585
762,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2019,NaN,3.0,278.9150,8415,6280487,0.133986
763,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2020,NaN,6.4,291.8200,8332,6366679,0.130869


#### Merge previous data with income data

In [24]:
# df = pd.merge(df, income_data, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')

In [25]:
df = pd.merge(df, disability, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df = df[["MSA_x", "Primary MSA City", "Primary MSA State", "Year", "Inflation Rate", "Unemployment Rate", "HPI (Not Seasonally Adjusted)", "Overall Homeless", "Population", "Homeless Rate (Percentage of Population)", "Percent With A Disability"]]
df

,MSA_x,Primary MSA City,Primary MSA State,Year,Inflation Rate,Unemployment Rate,HPI (Not Seasonally Adjusted),Overall Homeless,Population,Homeless Rate (Percentage of Population),Percent With A Disability
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2007,3.2,4.8,181.4050,6840,4939198,0.138484,NaN
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2008,2.9,6.7,174.9125,6840,5046727,0.135533,NaN
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2009,-2.5,10.4,164.4725,7019,5156598,0.136117,NaN
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2010,1.2,10.4,151.7375,7019,5268860,0.133217,9.1
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2011,2.9,9.8,140.0650,6805,5366462,0.126806,9.8
...,...,...,...,...,...,...,...,...,...,...,...
760,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2017,1.3,3.6,260.1925,9393,6211341,0.151223,9.0
761,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2018,NaN,3.3,268.7350,8846,6247841,0.141585,8.8
762,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2019,NaN,3.0,278.9150,8415,6280487,0.133986,8.8
763,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2020,NaN,6.4,291.8200,8332,6366679,0.130869,NaN


In [26]:
df = pd.merge(df, economic, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df = df[['MSA', 'Primary MSA City', 'Primary MSA State', 'Year',
       'Inflation Rate', 'Unemployment Rate',
       'HPI (Not Seasonally Adjusted)', 'Overall Homeless', 'Population',
       'Homeless Rate (Percentage of Population)',
       'Percent With A Disability', 'MSA',
       'Population (persons) 3/', 'Total employment (number of jobs)',
       'Average earnings per job (dollars)',
       'Average wages and salaries (dollars)',
       'Per capita net earnings 4/ (dollars)']]

In [27]:
df = pd.merge(df, uninsured, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df.rename(columns={"MSA_x": "MSA", "Percentage of Uninsured": "Percentage of Population without Health Insurance"}, inplace=True)
df

,MSA,Primary MSA City,Primary MSA State,Year,Inflation Rate,Unemployment Rate,HPI (Not Seasonally Adjusted),Overall Homeless,Population,Homeless Rate (Percentage of Population),Percent With A Disability,MSA,Population (persons) 3/,Total employment (number of jobs),Average earnings per job (dollars),Average wages and salaries (dollars),Per capita net earnings 4/ (dollars),MSA_y,Percentage of Population without Health Insurance
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2007,3.2,4.8,181.4050,6840,4939198,0.138484,NaN,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1550291.0,1377965.0,60308.50,54231.50,36294.00,NaN,NaN
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2008,2.9,6.7,174.9125,6840,5046727,0.135533,NaN,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1574340.0,1373823.0,58787.50,54938.50,33614.00,NaN,NaN
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2009,-2.5,10.4,164.4725,7019,5156598,0.136117,NaN,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1596169.0,1316341.0,59742.50,55239.50,33045.50,NaN,NaN
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2010,1.2,10.4,151.7375,7019,5268860,0.133217,9.1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1618043.0,1314438.0,60631.00,56502.50,33518.50,"Atlanta-Sandy Springs-Marietta, GA",19.855243
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2011,2.9,9.8,140.0650,6805,5366462,0.126806,9.8,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1645464.0,1356683.0,61490.50,58142.50,35332.00,"Atlanta-Sandy Springs-Marietta, GA",19.651024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2017,1.3,3.6,260.1925,9393,6211341,0.151223,9.0,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1471579.0,1501091.0,88303.00,81330.50,60149.25,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7.579970
761,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2018,NaN,3.3,268.7350,8846,6247841,0.141585,8.8,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1486036.0,1524121.0,91120.50,84751.50,62675.75,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7.515439
762,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2019,NaN,3.0,278.9150,8415,6280487,0.133986,8.8,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1499021.0,1535453.0,93425.25,86451.25,63774.75,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7.526310
763,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2020,NaN,6.4,291.8200,8332,6366679,0.130869,NaN,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1491461.0,1471123.0,98979.25,93441.50,64841.25,NaN,NaN


In [28]:
safety.rename(columns={"Primary City in MSA": "Primary MSA City", "Primary State in MSA": "Primary MSA State"}, inplace=True)

In [29]:
df = pd.merge(df, safety, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df.rename(columns={"MSA_x": "MSA"}, inplace=True)
df

,MSA,Primary MSA City,Primary MSA State,Year,Inflation Rate,Unemployment Rate,HPI (Not Seasonally Adjusted),Overall Homeless,Population,Homeless Rate (Percentage of Population),...,MSA,Population (persons) 3/,Total employment (number of jobs),Average earnings per job (dollars),Average wages and salaries (dollars),Per capita net earnings 4/ (dollars),MSA_y,Percentage of Population without Health Insurance,MSA_y,Total Year-Round Beds (ES+TH)
0,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2007,3.2,4.8,181.4050,6840,4939198,0.138484,...,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1550291.0,1377965.0,60308.50,54231.50,36294.00,NaN,NaN,"Atlanta-Sandy Springs-Alpharetta, GA MSA",4806.0
1,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2008,2.9,6.7,174.9125,6840,5046727,0.135533,...,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1574340.0,1373823.0,58787.50,54938.50,33614.00,NaN,NaN,"Atlanta-Sandy Springs-Alpharetta, GA MSA",4768.0
2,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2009,-2.5,10.4,164.4725,7019,5156598,0.136117,...,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1596169.0,1316341.0,59742.50,55239.50,33045.50,NaN,NaN,"Atlanta-Sandy Springs-Alpharetta, GA MSA",5254.0
3,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2010,1.2,10.4,151.7375,7019,5268860,0.133217,...,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1618043.0,1314438.0,60631.00,56502.50,33518.50,"Atlanta-Sandy Springs-Marietta, GA",19.855243,"Atlanta-Sandy Springs-Alpharetta, GA MSA",5004.0
4,"Atlanta-Sandy Springs-Alpharetta, GA MSA",Atlanta,GA,2011,2.9,9.8,140.0650,6805,5366462,0.126806,...,"Atlanta-Sandy Springs-Alpharetta, GA MSA",1645464.0,1356683.0,61490.50,58142.50,35332.00,"Atlanta-Sandy Springs-Marietta, GA",19.651024,"Atlanta-Sandy Springs-Alpharetta, GA MSA",5890.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2017,1.3,3.6,260.1925,9393,6211341,0.151223,...,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1471579.0,1501091.0,88303.00,81330.50,60149.25,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7.579970,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",9021.0
761,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2018,NaN,3.3,268.7350,8846,6247841,0.141585,...,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1486036.0,1524121.0,91120.50,84751.50,62675.75,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7.515439,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",8291.0
762,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2019,NaN,3.0,278.9150,8415,6280487,0.133986,...,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1499021.0,1535453.0,93425.25,86451.25,63774.75,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7.526310,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",7691.0
763,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",Washington,DC,2020,NaN,6.4,291.8200,8332,6366679,0.130869,...,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",1491461.0,1471123.0,98979.25,93441.50,64841.25,NaN,NaN,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA",8166.0


In [30]:
rentals.rename(columns={"Primary City in MSA": "Primary MSA City", "Primary State in MSA": "Primary MSA State"}, inplace=True)
df = pd.merge(df, rentals, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')
df.rename(columns={"MSA_x": "MSA"},inplace=True)
df.drop(columns=['MSA_y'], inplace=True)

In [31]:
df = pd.merge(df, snap, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')

In [32]:
deaths.rename(columns={"Primary City in MSA": "Primary MSA City", "Primary State in MSA": "Primary MSA State"}, inplace=True)
df = pd.merge(df, deaths, on=["Primary MSA City", "Primary MSA State", "Year"])

In [33]:
df = pd.merge(df, demographics, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')

In [34]:
income_data.rename(columns={"State": "Primary MSA State", "City": "Primary MSA City"}, inplace=True)

In [35]:
df.columns.values

array(['MSA_x', 'Primary MSA City', 'Primary MSA State', 'Year',
       'Inflation Rate', 'Unemployment Rate',
       'HPI (Not Seasonally Adjusted)', 'Overall Homeless',
       'Population_x', 'Homeless Rate (Percentage of Population)',
       'Percent With A Disability', 'MSA_x', 'Population (persons) 3/',
       'Total employment (number of jobs)',
       'Average earnings per job (dollars)',
       'Average wages and salaries (dollars)',
       'Per capita net earnings 4/ (dollars)',
       'Percentage of Population without Health Insurance',
       'Total Year-Round Beds (ES+TH)', 'Studio Rental Price',
       '1BR Rental Price', '2BR Rental Price', '3BR Rental Price',
       '4BR Rental Price', 'MSA_y', 'nSNAP', 'MSA', 'Population_y',
       'Overdose Deaths', 'Unintentional Overdose Deaths',
       'Alcohol-Induced Deaths', 'Population',
       'Sheltered Homeless_percentage',
       'Sheltered Homeless - Under 18_percentage',
       'Sheltered Homeless - Age 18 to 24_percentage

In [36]:
del df['MSA_x']
del df['MSA_y']
del df['Population_y']
del df['Population_x'],

df = pd.merge(df, income_data, on=["Primary MSA City", "Primary MSA State", "Year"], how='left')

df.rename({"Percentage of Uninsured": "Percentage of Population without Health Insurance"},inplace=True)
df.columns.values

array(['Primary MSA City', 'Primary MSA State', 'Year', 'Inflation Rate',
       'Unemployment Rate', 'HPI (Not Seasonally Adjusted)',
       'Overall Homeless', 'Homeless Rate (Percentage of Population)',
       'Percent With A Disability', 'Population (persons) 3/',
       'Total employment (number of jobs)',
       'Average earnings per job (dollars)',
       'Average wages and salaries (dollars)',
       'Per capita net earnings 4/ (dollars)',
       'Percentage of Population without Health Insurance',
       'Total Year-Round Beds (ES+TH)', 'Studio Rental Price',
       '1BR Rental Price', '2BR Rental Price', '3BR Rental Price',
       '4BR Rental Price', 'nSNAP', 'MSA_x', 'Overdose Deaths',
       'Unintentional Overdose Deaths', 'Alcohol-Induced Deaths',
       'Population', 'Sheltered Homeless_percentage',
       'Sheltered Homeless - Under 18_percentage',
       'Sheltered Homeless - Age 18 to 24_percentage',
       'Sheltered Homeless - Over 24_percentage',
       'Sheltered 

In [38]:
df

,Primary MSA City,Primary MSA State,Year,Inflation Rate,Unemployment Rate,HPI (Not Seasonally Adjusted),Overall Homeless,Homeless Rate (Percentage of Population),Percent With A Disability,Population (persons) 3/,...,Overall Homeless - Gender Non-Conforming_percentage,Overall Homeless - Hispanic/Latino_percentage,Overall Homeless - White_percentage,Overall Homeless - Black or African American_percentage,Overall Homeless - Asian_percentage,Overall Homeless - American Indian or Alaska Native_percentage,Overall Homeless - Native Hawaiian or Other Pacific Islander_percentage,Overall Homeless - Multiple Races_percentage,Median Income ($),MSA_y
0,Atlanta,GA,2007,3.2,4.8,181.4050,6840,0.138484,NaN,1550291.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
1,Atlanta,GA,2008,2.9,6.7,174.9125,6840,0.135533,NaN,1574340.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2,Atlanta,GA,2009,-2.5,10.4,164.4725,7019,0.136117,NaN,1596169.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
3,Atlanta,GA,2010,1.2,10.4,151.7375,7019,0.133217,9.1,1618043.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41631.0,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
4,Atlanta,GA,2011,2.9,9.8,140.0650,6805,0.126806,9.8,1645464.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43903.0,"Atlanta-Sandy Springs-Alpharetta, GA MSA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,Washington,DC,2016,1.2,3.8,252.2925,10341,0.168165,8.6,1447567.0,...,0.000000,0.012505,0.020067,0.141886,0.001577,0.000732,0.000342,0.003561,75506.0,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"
654,Washington,DC,2017,1.3,3.6,260.1925,9393,0.151223,9.0,1471579.0,...,0.000048,0.012928,0.018515,0.125351,0.001610,0.000531,0.000435,0.004782,82372.0,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"
655,Washington,DC,2018,NaN,3.3,268.7350,8846,0.141585,8.8,1486036.0,...,0.000160,0.009667,0.018758,0.117529,0.001633,0.000848,0.000384,0.002433,85203.0,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"
656,Washington,DC,2019,NaN,3.0,278.9150,8415,0.133986,8.8,1499021.0,...,0.000159,0.008805,0.016798,0.109976,0.001656,0.000828,0.000350,0.004379,92266.0,"Washington-Arlington-Alexandria, DC-VA-MD-WV MSA"


In [40]:
del df['MSA_y']

KeyError: ignored

In [41]:
df.to_csv("/content/drive/MyDrive/DATA 511 Final Project/output_v2.csv")